### Agents

In [1]:
import os,sys
sys.path.insert(0,'../../libs')
from utils import load_json

### Load all API keys 
openai_key = load_json('/home/chengyu.huang/project/Fund_projects/openai_key.json') 
os.environ['OPENAI_API_KEY'] = openai_key['ChatGPT']['API_KEY']

### initiate a llm

In [2]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],  # platform.openai.com
    temperature=0,
    model_name="gpt-3.5-turbo"
)

/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/langchain/llms/openai.py:739: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


#### Initialize a calculator tool using LLMMathChain

In [3]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [4]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [5]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

#### intiate a zero shoot react agent to choose whcih tool to use 

In [6]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",   ## for now, it doesn't have memory, only use description to do the filtering 
	tools=tools,
	llm=llm,
	verbose=True,        ## set it to false, we can get output directly 
	max_iterations=3     ## limit it to max loop in case it get stucked 
)

In [7]:
## give it a simple try 
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to use the calculator to solve this problem.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

- a slightly more complicated problem 

In [8]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
We need to convert the apple boxes to the number of apples they contain.
Action: Calculator
Action Input: 2.5 x 8
Observation: Answer: 20.0
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 5904a56c90a266cbb216338fc2cf6088 in your message.).


Now we add the number of apples Giorgio brought to the number of apples Mary had.
Action: Calculator
Action Input: 4 + 20
Observation: Answer: 24
Thought:The final answer is 24 apples. 
Final Answer: 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': '24 apples.'}

A more common case is we add a basic llm chain and plus all other tools, so that if additional tools are not useful, it will default to use llm 

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)
     

In [12]:
## add llm tool to tool list 
tools.append(llm_tool)

## initiate the zero shot agent again 
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)


In [13]:
## now try it out 
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
I don't know the answer to this question off the top of my head, so I should use a language model to search for it.
Action: Language Model
Action Input: "What is the capital of Norway?"
Observation: The capital of Norway is Oslo.
Thought:I now know the answer to the question.
Final Answer: Oslo is the capital of Norway.

> Finished chain.


{'input': 'what is the capital of Norway?',
 'output': 'Oslo is the capital of Norway.'}

In [14]:
## give it a simple try 
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I should use the calculator for this math problem.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the answer to the math problem.
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

#### There are a lot prebuilt tools in langchain

In [20]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [21]:
tools[0].name,tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

## What is an agent?
**Definition**: The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:


> Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In [24]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [22]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Language Model: use this tool for general purpose queries and logic
Language Model: use this tool for general purpose queries and logic

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Language Model, Language Model]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### see [here](https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/06-langchain-agents.ipynb) for an example of combining sql agent together with calculator to perform a task

Suffice it to say for now that **the LLM now has the ability to 'reason' on how to best use tools** to solve our query and can combine them in intelligent ways with just a brief description of each of them. If you want to learn more about this paradigm (MRKL) in detail, please refer to [this](https://arxiv.org/pdf/2205.00445.pdf) paper. 

Finally, let's pay attention to the 'agent_scratchpad'. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

### Agent type #2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [25]:
result = count_tokens(
    conversational_agent, 
    "Please provide me the stock prices for ABC on January the 1st"
                     )

## we did not add db query tool here, so it won't work. 



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: I'm sorry, but I don't have access to real-time stock prices. However, you can check financial news websites or use a stock market app to get the latest stock prices for ABC on January 1st.

> Finished chain.
Spent a total of 446 tokens
